In [1]:
import numpy as np
import sys
sys.path.append('../')
from utils.sparse_vector import *
from utils.vector_polynomial_isomorphism import *
from utils.minor_flattening import *

In [2]:
# Let us initialize constants and vector spaces.

N = 2*2*3*3
n = [2,2,3,3]
d = 6
indeterminates = form_indeterminates(n)
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)

Defining x_0000, x_0001, x_0002, x_0010, x_0011, x_0012, x_0020, x_0021, x_0022, x_0100, x_0101, x_0102, x_0110, x_0111, x_0112, x_0120, x_0121, x_0122, x_1000, x_1001, x_1002, x_1010, x_1011, x_1012, x_1020, x_1021, x_1022, x_1100, x_1101, x_1102, x_1110, x_1111, x_1112, x_1120, x_1121, x_1122


In [3]:
f = get_flattening_minors(R, n, d)
monomial_to_index = {}

for k in f[0].monomials():
    if k not in monomial_to_index:
        monomial_to_index[k] = len(monomial_to_index)
        
for k in f[1].monomials():
    if k not in monomial_to_index:
        monomial_to_index[k] = len(monomial_to_index)

index_to_monomial = list(monomial_to_index.keys())

dim_V_small = len(monomial_to_index)
V_small = VectorSpace(QQ, dim_V_small, sparse=True)
f1_vec = [0]*dim_V_small
f2_vec = [0]*dim_V_small

for mon in f[0].monomials():
    i = monomial_to_index[mon]
    f1_vec[i] = f[0].coefficient(mon)
    
for mon in f[1].monomials():
    i = monomial_to_index[mon]
    f2_vec[i] = f[1].coefficient(mon)

f1_vec = V_small(f1_vec)
f2_vec = V_small(f2_vec)


W = V_small.subspace([f1_vec, f2_vec])

In [4]:
W_perp = W.basis_matrix().right_kernel_matrix(algorithm='default', basis='computed')

p = W.dimension()
q = W_perp.nrows()

In [5]:
def E_action(a, b, grad_fl):
    sumh = -1*indeterminates[b]*grad_fl[a]
    return R(sumh)

grad_fl = [f[0].gradient(), f[1].gradient()]

for l in range(2):
    E_action_fl = [[0 for j in range(N)] for i in range(N)]
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = E_action(i,j,grad_fl[l])
            for k in E_action_fl[i][j].monomials():
                if k not in monomial_to_index:
                    monomial_to_index[k] = len(monomial_to_index)
                    
index_to_monomial = list(monomial_to_index.keys())
print(len(monomial_to_index))

227664


In [6]:
g_sparse = [sparsify(W_perp[k]) for k in range(q)]

In [7]:
for i in range(1294, len(monomial_to_index)):
    v = [(i, 1)]
    g_sparse.append(v)
    
q = len(g_sparse)

In [11]:
def poly_to_sparsevec(p):
    ans = []
    for mon in p.monomials():
        ans.append((monomial_to_index[mon], p.monomial_coefficient(mon)))
    return ans

E_action_fl = [[0 for j in range(N)] for i in range(N)]

M_f = zero_matrix(QQ, p*q, N^2, sparse = True)
for l in range(p):
    grad_fl = f[l].gradient()
    print(l)
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = poly_to_sparsevec(E_action(i,j,grad_fl))
    print('Eaction computed')
    for k in range(q):
        if k % 1000 == 0:
            print(k)
        for i in range(N):
            for j in range(N):
                M_f[(q*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse[k])

0
Eaction computed
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
15

In [12]:
rank = M_f.rank()

print('rank(M_f) =', rank)
print('dim(ker(M_f)) =', glN.dimension() - rank)

rank(M_f) = 1292
dim(ker(M_f)) = 4


In [13]:
print('expected:', sum([ni^2-1 for ni in n]) + 1)

expected: 23
